# 01. Time Travel

Iceberg의 **Time Travel** 기능을 활용하여 과거 시점의 데이터를 조회하고, 실수로 삭제된 데이터를 복구합니다.

## Time Travel 개념

Iceberg는 모든 데이터 변경을 **스냅샷(Snapshot)**으로 관리합니다.

```
Snapshot 1 (INSERT 100건)
    │
    ▼
Snapshot 2 (UPDATE 20건)
    │
    ▼
Snapshot 3 (DELETE 10건)
    │
    ▼
Snapshot 4 (INSERT 50건)  ◀ current
```

- 각 스냅샷은 **불변(immutable)**하며, 이전 스냅샷의 데이터 파일은 그대로 유지됩니다
- `VERSION AS OF` 또는 `TIMESTAMP AS OF`로 과거 시점의 데이터를 조회할 수 있습니다
- `rollback_to_snapshot`으로 특정 스냅샷 상태로 테이블을 되돌릴 수 있습니다

### Time Travel의 활용 사례

| 시나리오 | 방법 |
|----------|------|
| 과거 시점 데이터 분석 | `SELECT * FROM table VERSION AS OF {snapshot_id}` |
| 시간 기반 데이터 조회 | `SELECT * FROM table TIMESTAMP AS OF '{timestamp}'` |
| 실수 복구 (잘못된 DELETE/UPDATE) | `CALL system.rollback_to_snapshot(...)` |
| 데이터 변경 추적 (audit) | 스냅샷별 데이터 비교 |

## 환경 설정

In [ ]:
import sys
sys.path.append('..')

from utils.spark_setup import create_spark_session
from utils.data_generator import generate_orders, to_spark_df

In [ ]:
spark = create_spark_session("TimeTravel")

## 테이블 생성 및 4개 스냅샷 만들기

In [ ]:
spark.sql("CREATE DATABASE IF NOT EXISTS demo.lab")
spark.sql("DROP TABLE IF EXISTS demo.lab.tt_orders")

spark.sql("""
    CREATE TABLE demo.lab.tt_orders (
        order_id     BIGINT,
        customer_id  BIGINT,
        product_name STRING,
        order_date   DATE,
        amount       DOUBLE,
        status       STRING
    )
    USING iceberg
    PARTITIONED BY (months(order_date))
""")

print("테이블 생성 완료")

In [ ]:
# Snapshot 1: 100건 INSERT
orders1 = generate_orders(num_records=100, seed=42)
df1 = to_spark_df(spark, orders1)
df1.writeTo("demo.lab.tt_orders").append()

count = spark.sql("SELECT COUNT(*) AS cnt FROM demo.lab.tt_orders").collect()[0]["cnt"]
print(f"Snapshot 1 (INSERT): {count}건")

In [ ]:
# Snapshot 2: 20건 UPDATE
spark.sql("""
    UPDATE demo.lab.tt_orders
    SET status = 'cancelled', amount = 0.0
    WHERE order_id <= 20
""")

count = spark.sql("SELECT COUNT(*) AS cnt FROM demo.lab.tt_orders").collect()[0]["cnt"]
cancelled = spark.sql("SELECT COUNT(*) AS cnt FROM demo.lab.tt_orders WHERE status = 'cancelled'").collect()[0]["cnt"]
print(f"Snapshot 2 (UPDATE): 총 {count}건, cancelled={cancelled}건")

In [ ]:
# Snapshot 3: 10건 DELETE
spark.sql("""
    DELETE FROM demo.lab.tt_orders
    WHERE order_id BETWEEN 21 AND 30
""")

count = spark.sql("SELECT COUNT(*) AS cnt FROM demo.lab.tt_orders").collect()[0]["cnt"]
print(f"Snapshot 3 (DELETE): {count}건")

In [ ]:
# Snapshot 4: 50건 INSERT
orders2 = generate_orders(num_records=50, id_offset=101, seed=99)
df2 = to_spark_df(spark, orders2)
df2.writeTo("demo.lab.tt_orders").append()

count = spark.sql("SELECT COUNT(*) AS cnt FROM demo.lab.tt_orders").collect()[0]["cnt"]
print(f"Snapshot 4 (INSERT): {count}건")

## 스냅샷 목록 확인

In [ ]:
snapshots_df = spark.sql("SELECT * FROM demo.lab.tt_orders.snapshots")
snapshots_df.show(truncate=False)

In [ ]:
# 스냅샷 ID 목록 저장
snapshot_rows = snapshots_df.collect()
snapshot_ids = [row["snapshot_id"] for row in snapshot_rows]

print("스냅샷 ID 목록:")
for i, sid in enumerate(snapshot_ids):
    print(f"  Snapshot {i+1}: {sid}")

## VERSION AS OF로 과거 시점 데이터 조회

각 스냅샷 시점의 데이터 수를 비교합니다.

In [ ]:
print("[ 스냅샷별 레코드 수 비교 ]\n")

for i, sid in enumerate(snapshot_ids):
    count = spark.sql(f"""
        SELECT COUNT(*) AS cnt
        FROM demo.lab.tt_orders
        VERSION AS OF {sid}
    """).collect()[0]["cnt"]
    
    operations = ["INSERT 100건", "UPDATE 20건", "DELETE 10건", "INSERT 50건"]
    op = operations[i] if i < len(operations) else "?"
    print(f"  Snapshot {i+1} ({op}): {count}건")

In [ ]:
# Snapshot 1의 데이터 샘플 조회
print(f"[ Snapshot 1 데이터 샘플 (ID: {snapshot_ids[0]}) ]\n")

spark.sql(f"""
    SELECT order_id, product_name, amount, status
    FROM demo.lab.tt_orders
    VERSION AS OF {snapshot_ids[0]}
    ORDER BY order_id
    LIMIT 10
""").show(truncate=False)

## TIMESTAMP AS OF로 시간 기반 조회

In [ ]:
# 각 스냅샷의 타임스탬프 확인
history_df = spark.sql("""
    SELECT snapshot_id, committed_at
    FROM demo.lab.tt_orders.snapshots
    ORDER BY committed_at
""")
history_df.show(truncate=False)

In [ ]:
# 첫 번째 스냅샷의 타임스탬프로 Time Travel
first_ts = snapshot_rows[0]["committed_at"]
ts_str = str(first_ts)

print(f"Timestamp: {ts_str}\n")

count = spark.sql(f"""
    SELECT COUNT(*) AS cnt
    FROM demo.lab.tt_orders
    TIMESTAMP AS OF '{ts_str}'
""").collect()[0]["cnt"]

print(f"TIMESTAMP AS OF '{ts_str}' → {count}건")

## 실수 복구 시나리오

실수로 테이블의 **전체 데이터를 삭제**한 뒤, `rollback_to_snapshot`으로 복구합니다.

### 시나리오
1. 현재 상태 확인
2. 실수로 `DELETE FROM ... WHERE 1=1` 실행 (전체 삭제!)
3. 데이터가 0건인 것을 확인
4. 스냅샷 기반 롤백으로 복구

In [ ]:
# 1. 현재 상태
before_count = spark.sql("SELECT COUNT(*) AS cnt FROM demo.lab.tt_orders").collect()[0]["cnt"]
print(f"삭제 전 레코드 수: {before_count}건")

# 복구할 스냅샷 ID 미리 저장 (현재 스냅샷 = Snapshot 4)
recovery_snapshot_id = snapshot_ids[-1]
print(f"복구 대상 스냅샷 ID: {recovery_snapshot_id}")

In [ ]:
# 2. 실수로 전체 삭제! 😱
spark.sql("DELETE FROM demo.lab.tt_orders WHERE 1=1")

after_count = spark.sql("SELECT COUNT(*) AS cnt FROM demo.lab.tt_orders").collect()[0]["cnt"]
print(f"삭제 후 레코드 수: {after_count}건 — 데이터가 모두 사라졌습니다!")

In [ ]:
# 3. rollback_to_snapshot으로 복구
print(f"Snapshot {recovery_snapshot_id}로 롤백합니다...\n")

spark.sql(f"""
    CALL demo.system.rollback_to_snapshot('demo.lab.tt_orders', {recovery_snapshot_id})
""")

recovered_count = spark.sql("SELECT COUNT(*) AS cnt FROM demo.lab.tt_orders").collect()[0]["cnt"]
print(f"복구 후 레코드 수: {recovered_count}건")
print(f"\n복구 성공! ({after_count}건 → {recovered_count}건)")

In [ ]:
# 복구 후 스냅샷 목록 확인
print("[ 복구 후 스냅샷 목록 ]\n")
spark.sql("SELECT * FROM demo.lab.tt_orders.snapshots").show(truncate=False)

## 관찰 포인트

### Time Travel 핵심 정리

1. **스냅샷 기반 버전 관리**: 모든 작업은 새로운 스냅샷을 생성하며, 이전 스냅샷은 그대로 유지됩니다

2. **데이터 조회 방법**:
   - `VERSION AS OF {snapshot_id}`: 특정 스냅샷의 데이터
   - `TIMESTAMP AS OF '{timestamp}'`: 특정 시점의 데이터

3. **실수 복구**: `rollback_to_snapshot`으로 전체 삭제도 복구 가능
   - 롤백 자체도 새로운 스냅샷을 생성합니다
   - 기존 스냅샷은 삭제되지 않습니다

### ⚠️ 주의사항

- **스냅샷이 있는 한** 데이터를 복구할 수 있습니다
- 그러나 `expire_snapshots`를 실행하면 오래된 스냅샷이 삭제되고, 해당 스냅샷으로의 Time Travel은 **불가능**해집니다
- 스냅샷 만료는 스토리지 절약을 위해 주기적으로 실행하되, 복구 가능 기간을 고려하여 설정해야 합니다
- 이 부분은 `4_optimization`에서 더 자세히 다룹니다

In [ ]:
spark.stop()
print("Spark 세션 종료")